In [ ]:
# name: arojas
# date: 3/20/2020
# file: riemann_arcpy
# tests riemann sum on sample raster using arcpy functionality

We can use the example to generate a random raster from the ArcGIS Pro [website](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/create-random-raster.htm). First I want to create a test raster and some random points on which I'll create buffers. These buffers will be the areas on which I'll perform the Riemann sum. 

I don't have direct access to ArcGIS because of the COVID-19 situation on campus, so I am trying my best to do this through UNC's Virtual Lab. Once I can get access to IDLE + the ArcGIS dependencies, I can make this code look a lot cleaner, e.g. organizing the `arcpy.env.workspace` so I stop repeating things unnecessarily. For now, this code is functional and serves this temporary purpose. 

In [ ]:
# name: arojas
# date: 3/20/2020
# file: riemann_arcpy
# tests riemann sum on sample raster using arcpy functionality

import arcpy
 
# define variables
arcpy.env.workspace = "T:\\Students\\alfredoj\\final_project\\output"
arcpy.env.overwriteOutput = True # overwrite output

# out_path = "output"
outFile = "test.tif"
distribution = "POISSON 6.4"
outExtent = "250 250 750 750"
cellSize = 10
 
# Execute CreateRandomRaster
arcpy.CreateRandomRaster_management(arcpy.env.workspace, outFile, distribution, outExtent, cellSize)

# Create random points in the features of a constraining feature class
points = "random_points.shp"
con_extent = "test.tif" # may not need the absolute path
arcpy.CreateRandomPoints_management(arcpy.env.workspace, points, "", con_extent, 10)

# input/output data
input_ras = "test.tif"
input_pts = "random_points.shp"
output_ras = "test_Proj2.tif"
output_pts = "pts_Proj2.shp"

# create a spatial reference object for the output coordinate system
out_CRS = "PROJCS['NAD_1983_UTM_Zone_1N',GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],PROJECTION['Transverse_Mercator'],PARAMETER['False_Easting',500000.0],PARAMETER['False_Northing',0.0],PARAMETER['Central_Meridian',-177.0],PARAMETER['Scale_Factor',0.9996],PARAMETER['Latitude_Of_Origin',0.0],UNIT['Meter',1.0]]"

# run the projection tools
arcpy.ProjectRaster_management(input_ras, output_ras, out_CRS, "NEAREST", "", "", "", out_CRS)
arcpy.Project_management(input_pts, output_pts, out_CRS, "", out_CRS, "NO_PRESERVE_SHAPE", "", "NO_VERTICAL")

# Crate buffer around points
out_buff = "pt_buffer.shp"
arcpy.Buffer_analysis(points, out_buff, "20 Meters", "FULL", "ROUND", "NONE")

fields = ["FID"]
cursor = arcpy.da.SearchCursor(out_buff, fields)
for item in cursor:
    print(item[0])

# get raster cell size, make sure to square
x = arcpy.GetRasterProperties_management("test.tif", "CELLSIZEX")
int(x[0])

# Import system modules
import arcpy
from arcpy.sa import *

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

out_zonal = arcpy.ZonalStatistics("pt_buffer.shp", "Shape", "test.tif", "SUM")

# some kind of cursor to modify buffer attribute table
# cursor to read attribute table

# For every buffer
    # extract raster by mask
    # zonal stats add all values, multiply by resolution
    # add value to buffer attribute table. . . 

# Replace a layer/table view name with a path to a dataset (which can be a layer file) or create the layer/table view within the script
# The following inputs are layers or table views: "test.tif"
